In [33]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from operator import add

In [2]:
conf = SparkConf().set("spark.ui.port", "4050")

# creamos el contexto y la sesión
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/15 04:20:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
dataRDD = sc.textFile('./data/paises.csv').map(lambda x: x.split(','))
dataRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

#### Número de países participantes

In [5]:
dataRDD.map(lambda x: (x[2])).distinct().count()

231

#### Número de equipos por país

In [6]:
dataRDD.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(15)

[('ARG', 18),
 ('AFG', 1),
 ('IRL', 7),
 ('POR', 21),
 ('AUS', 23),
 ('USA', 92),
 ('ANG', 1),
 ('DEN', 28),
 ('ANT', 1),
 ('CAN', 31),
 ('GER', 40),
 ('GRE', 28),
 ('ANZ', 2),
 ('AZE', 1),
 ('BIZ', 1)]

In [7]:
dataRDD.map(lambda x: (x[2], x[1])).groupByKey().mapValues(list).take(5)

[('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan']),
 ('IRL',
  ['Akatonbo',
   'Aletta',
   'Ireland',
   'Ireland-1',
   'Ireland-2',
   'The Cloud',
   'Three Leaves']),
 ('POR',
  ['Alcaid',
   'Argus',
   'Calcinhas',
   'Camelia',
   'Ciocca III',
   'Espadarte',
   'Espardate',
   'Faneca',
   'Galopin De La Font',
   'Grifo III',
   'Grifo IV',
   'Hannover',
   "Ma'Lindo",
   'Notavel',
   'Oxalis',
   'Portugal',
   'Portugal-1',
   'Portugal-2',
   'Sjhxa',
   'Symphony',
   'Vicking']),
 ('AUS',
  ['Amateur Athletic Association',
   'Australia',
   'Australia-1',
   'Australia-2',
   'Australia-3',
   'Australia/Great Britain',
   'Barrenjoey',
   'Buraddoo',
   'Cambria',
   'Diablo',
   'Falcon IV',
   'Falcon VI',
   'Ga

#### Filtrar los equipos argentinos

In [8]:
teamsARG = dataRDD.filter(lambda x: "ARG" in x).collect()
teamsARG

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [9]:
dataRDD.count()

1185

In [10]:
dataRDD.countApprox(20)

1185

#### Concatenar RDDs

In [11]:
player1 = sc.textFile('./data/player.csv').map(lambda x: x.split(','))
player2 = sc.textFile('./data/player2.csv').map(lambda x: x.split(','))
player1.take(3)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199']]

In [12]:
player2.take(3)

[['67787', 'Lee BongJu', '1', '27', '167', '56', '970'],
 ['67788', 'Lee BuTi', '1', '23', '164', '54', '203'],
 ['67789', 'Anthony N. Buddy Lee', '1', '34', '172', '62', '1096']]

In [13]:
playerRDD = player1.union(player2)
playerRDD.count()

135572

In [14]:
playerRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [15]:
dataRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

#### Join entre RDDs

In [16]:
playerRDD.map(lambda x: [x[-1], x[:-1]])\
            .join(dataRDD.map(lambda x: [x[0], x[-1]]))\
            .take(6)

[('278', (['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0'], 'SWE')),
 ('278', (['86368', 'August Nilsson', '1', '27', '0', '0'], 'SWE')),
 ('278', (['112720', 'Gustaf Fredrik Sderstrm', '1', '34', '0', '0'], 'SWE')),
 ('278',
  (['114107', 'Karl Gustaf Vilhelm Staaf Johansson ', '1', '19', '0', '0'],
   'SWE')),
 ('278',
  (['130945', 'Charles Gustav Wilhelm Winckler', '1', '33', '0', '0'], 'SWE')),
 ('982', (['51', 'Nstor Abad Sanjun', '1', '23', '167', '64'], 'ESP'))]

Tomar una muestra con datos sin repetir `False`, número de ejemplos `6` y un random state `25`

In [17]:
playerRDD.map(lambda x: [x[-1], x[:-1]])\
            .join(dataRDD.map(lambda x: [x[0], x[-1]]))\
            .takeSample(False, 6, 25)

[('514', (['58198', 'Mayumi Kawasaki', '2', '24', '167', '52'], 'JPN')),
 ('399',
  (['129907', 'Armin Christian Weyrauch', '1', '28', '193', '95'], 'GER')),
 ('487', (['53275', 'Khashaba Dadasaheb Jahdav', '1', '21', '0', '0'], 'IND')),
 ('138',
  (['110517', 'Hlio Frederico Gomes da Silva', '1', '22', '183', '79'],
   'BRA')),
 ('507', (['121829', 'Nicola Trentin', '1', '30', '178', '70'], 'ITA')),
 ('138',
  (['23297', 'Kissya Cataldo da Costa', '2', '30', '180', '67'], 'BRA'))]

In [20]:
playerTeamRDD = playerRDD.map(lambda x: [x[-1], x[:-1]])\
            .join(dataRDD.map(lambda x: [x[0], x[-1]]))
playerTeamRDD.take(2)

[('278', (['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0'], 'SWE')),
 ('278', (['86368', 'August Nilsson', '1', '27', '0', '0'], 'SWE'))]

#### Cargar `resultados.csv`

In [18]:
resultadosRDD = sc.textFile('./data/resultados.csv').map(lambda x: x.split(','))
resultadosRDD.take(3)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['1', 'NA', '1', '39', '1'],
 ['2', 'NA', '2', '49', '2']]

#### Filtrar por resultados con medalla

In [19]:
resultadosfRDD = resultadosRDD.filter(lambda x: "NA" not in x[1])
resultadosfRDD.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [21]:
playerCountryRDD = playerRDD\
    .map(lambda x: [x[-1], x[:-1]])\
    .join(dataRDD.map(lambda x: [x[0], x[2]]))
playerCountryRDD.take(3)

[('278', (['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0'], 'SWE')),
 ('278', (['86368', 'August Nilsson', '1', '27', '0', '0'], 'SWE')),
 ('278', (['112720', 'Gustaf Fredrik Sderstrm', '1', '34', '0', '0'], 'SWE'))]

In [22]:
playerCountryRDD.join(resultadosfRDD).take(3)

[('972',
  ((['3585', 'An JaeHyeong', '1', '23', '183', '74'], 'KOR'), 'Silver')),
 ('972', ((['19788', 'Chang YeNa', '2', '26', '172', '61'], 'KOR'), 'Silver')),
 ('972',
  ((['40891', 'Go SeongHyeon', '1', '25', '182', '85'], 'KOR'), 'Silver'))]

### Operaciones Numéricas

In [24]:
valsMedals = {
    'Gold': 7,
    'Silver': 5,
    'Bronze': 4
}

In [27]:
countryMedals = playerCountryRDD.join(resultadosfRDD)

In [29]:
countryMedals.map(lambda x: [x[1][0][-1], x[1][1]]).take(4)

[['KOR', 'Silver'], ['KOR', 'Silver'], ['KOR', 'Silver'], ['KOR', 'Silver']]

In [35]:
countryMedalsNum = countryMedals\
    .map(lambda x: [x[1][0][-1], valsMedals[x[1][1]]])
countryMedalsNum.take(3)

[['KOR', 5], ['KOR', 5], ['KOR', 5]]

In [37]:
finalResult = countryMedalsNum\
    .reduceByKey((add))\
    .sortBy(lambda x: x[1], ascending=False)
finalResult.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]